# Проект для «Викишоп» с BERT
Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

## План работ


1.   Загрузить и подготовить данные
2.   Обучить модели
3. Сделать итоговые выводы



In [1]:
!pip install catboost -q

In [2]:
!pip install transformers -q

In [49]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.dummy import DummyClassifier
from catboost import CatBoostClassifier

## 1 Загрузка и подготовка данных

Загружаю данные, вывожу информацию о датасете и первые 5 строк для ознакомления

In [4]:
try:
  data = pd.read_csv('/content/toxic_comments.csv')
except:
  data = pd.read_csv('/datasets/toxic_comments.csv')

In [5]:
data.info()
data.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


В данных нет пропусков, комментарии на анлгийском языке, это значит, что нужно будет использовать предобученную модель на английском

Подготавливаю данные для обучения

В силу технических ограничений в проекте буду использовать предобученую модель DistilBert. Так же для дообучения модели возьму только часть данных, в целях снижения времени обучения

In [6]:
data_for_bert = data.sample(5000)

In [7]:
text = data_for_bert['text'].tolist()
labels = data_for_bert['toxic'].tolist()

In [8]:
X_train, X_test, y_train, y_test = train_test_split(text, labels, test_size=0.2, random_state=42, stratify=labels)

Выполяю токенизацию данных с помощью предобученной модели для английского языка

In [16]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=64)
test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=64)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

## 2 BERT

Создаю кастомный класс, который будет использоваться как контейнер для данных. Создаю два набора данных (обучающий и тестовый)

In [10]:
class CustomDataset(Dataset):
      def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

      def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

      def __len__(self):
          return len(self.labels)

train_dataset = CustomDataset(train_encodings, y_train)
test_dataset = CustomDataset(test_encodings, y_test)

Загружаю предобученную модель и указываю количество классов

In [17]:
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=32,
    eval_strategy="epoch",
    save_strategy="epoch",
    report_to="none"
)

In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

Выполняю дообучение модели

In [25]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.098173
2,No log,0.135105
3,No log,0.149155


TrainOutput(global_step=375, training_loss=0.085925048828125, metrics={'train_runtime': 3836.034, 'train_samples_per_second': 3.128, 'train_steps_per_second': 0.098, 'total_flos': 198701097984000.0, 'train_loss': 0.085925048828125, 'epoch': 3.0})

Нахожу получившуюся метрику F1

In [27]:
preds = trainer.predict(test_dataset).predictions.argmax(-1)
f1 = f1_score(y_test, preds, average='binary')

In [31]:
print(f'Метрика f1_score равна: {f1}')

Метрика f1_score равна: 0.8


## 3 Модель CatBoost

Подготавливаю данные для обучения новой модели

In [32]:
X = data['text']
y = data['toxic']

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

Провожу векторизацию текста

In [35]:
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

Инициализирую и обучаю новую модель

In [40]:
model_cat = CatBoostClassifier(iterations=100, verbose=0)
model_cat.fit(X_train_vec, y_train)

In [41]:
preds_cat = model_cat.predict(X_test_vec)

Вычисляю метрику для CatBoostClassifier

In [48]:
print(f"Метрика f1 модели CatBoostClassifier: {f1_score(y_test, preds_cat, average='binary')}")

Метрика f1 модели CatBoostClassifier: 0.7526457399103139


Выполняю проверку работы модели с помощью сравнения метрики с dummy предсказанием

In [51]:
dummy = DummyClassifier(strategy="stratified")
dummy.fit(X_train_vec, y_train)


dummy_preds = dummy.predict(X_test_vec)


print(f"Dummy предсказание f1: {f1_score(y_test, dummy_preds, average='binary'):.4f}")

Dummy предсказание f1: 0.1016


У модели CatBoostClassifier метрика вышла намного лучше, чем у Dummy, это говорит о том, что модель работает хорошо

## Общий вывод

Модель DistilBert в ходе дообучения показала лучшие результаты метрики f1. Но эта модель гораздо требовательнее к технической части, чем CatBoost. По итогу разница не такая большая и в работе лучше использовать CatBoostClassifier, если скорость оценки комментариев важнее, чем высокая точность.